In [1]:
import math
from dataclasses import dataclass

@dataclass
class ErlangCResult:
    prob_wait: float   # P(wait > 0)
    asa: float         # Average Speed of Answer (seconds)
    occupancy: float   # Utilization (0-1)


def erlang_c(N: int, arrival_rate: float, aht: float) -> ErlangCResult:
    """
    Compute Erlang-C performance metrics.

    Parameters
    ----------
    N : int
        Number of agents (servers).
    arrival_rate : float
        Arrival rate λ (calls per second).
    aht : float
        Average handle time H (seconds).

    Returns
    -------
    ErlangCResult
        prob_wait: P(call has to wait)
        asa: Average speed of answer (seconds)
        occupancy: Agent occupancy (0-1)
    """
    if N <= 0:
        raise ValueError("N must be positive.")

    # Offered load in Erlangs: a = λ * H
    a = arrival_rate * aht

    # If offered load >= agents, system is unstable → infinite queue
    if a >= N:
        return ErlangCResult(
            prob_wait=1.0,
            asa=float("inf"),
            occupancy=1.0
        )

    # Denominator sum: sum_{k=0}^{N-1} a^k / k!
    denom_sum = 0.0
    for k in range(N):
        denom_sum += a**k / math.factorial(k)

    # "Last term" with queue correction: (a^N / N!) * (N / (N - a))
    last_term = (a**N / math.factorial(N)) * (N / (N - a))

    # Probability that all agents are busy (Erlang C wait probability)
    prob_wait = last_term / (denom_sum + last_term)

    # Average waiting time in queue (Wq)
    # Wq = Pw * H / (N - a)
    asa = (prob_wait * aht) / (N - a)

    # Occupancy = a / N
    occupancy = a / N

    return ErlangCResult(
        prob_wait=prob_wait,
        asa=asa,
        occupancy=occupancy
    )
